In [8]:
def TB(cleanup=False):
    if cleanup:
        !rm -R graphs/
        return
    import webbrowser
    webbrowser.open('http://127.0.1.1:6006')

    !tensorboard --logdir="./graphs"

In [9]:
import tensorflow as tf
saver = tf.train.Saver()
sess = tf.InteractiveSession()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [10]:
# Defining necessary functions

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)


def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)


def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


In [11]:
# First Convolutional Layer in 5x5x1 out 32

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# reshape input from 764 to 28x28

x_image = tf.reshape(x, [-1,28,28,1])

#rectified linear output & max pooling

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# 2nd layer 5x5x32 out 64

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

#rectified linear output & max pooling

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


# full connected layer

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# flatten last convolutional output,  pass it to full connected Layer
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#add dropout after full connected layer

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
y_ = tf.placeholder(tf.float32, [None, 10])


In [12]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [13]:


sess.run(tf.initialize_all_variables())
writer = tf.summary.FileWriter('./graphs',tf.get_default_graph())

for i in range(400):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    save_path = saver.save(sess, "./log/model_"+str(i)+".ckpt")
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

writer.close()



step 0, training accuracy 0.08
step 100, training accuracy 0.76
step 200, training accuracy 0.86
step 300, training accuracy 0.9


In [ ]:
TB()

W0620 19:37:26.841103 Reloader plugin_event_accumulator.py:283] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0620 19:37:26.842362 Reloader plugin_event_accumulator.py:283] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0620 19:37:26.842943 Reloader plugin_event_accumulator.py:291] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
TensorBoard 1.9.0a20180603 at http://hitesh:6006 (Press CTRL+C to quit)


In [5]:
TB(1)